In [ ]:
from sklearn.model_selection import ParameterGrid

%matplotlib inline
import shutil
import os
if os.path.exists('runs'):
    shutil.rmtree('runs')
input_shape = (96, 96)
cortex_shape = (96, 96)
param_grid = {'min_theta': [0, 0.2, 0.5], 'max_theta': [0.6, 0.8, 1.0], 'afferent_radius': [5, 10, 15, 20],
                  'excitatory_radius': [2, 5, 10, 15], 'inhibitory_radius': [5, 10, 15, 20, 25],
                  'settling_steps': [10], 'inhib_factor': [1, 1.2, 1.5, 2.0, 3], 'excit_factor': [1, 1.2, 1.5, 2.0, 3]}
param_grid = ParameterGrid(param_grid)

bar = generate_horizontal_bar(96)
activations = []
if not os.path.exists('act_npy'):
    os.mkdir('act_npy')

for v1_params in tqdm.tqdm_notebook(ParameterGrid(param_grid)):
    lissom, optimizer, _ = get_cortex(input_shape, cortex_shape, v1_params=v1_params)
    lissom.cuda()
    inp = torch.autograd.Variable(torch.from_numpy(bar))
    inp = Pipeline.process_input(inp)
    inp = inp.cuda()
    act = lissom(inp)
    activations.append(act.data)
    act_npy = np.reshape(act.data.cpu().numpy(), cortex_shape)
    from frozendict import frozendict
    np.save('act_npy/{}.npy'.format(str(hash(frozendict(v1_params)))), act_npy)
#     plt.imshow(act_npy, cmap='gray')
#     plt.show()


def load_img(v1_params):
    from frozendict import frozendict
    try:
        print("MAJOLA")
        img = np.load('act_npy/{}.npy'.format(str(hash(frozendict(v1_params)))))
        plt.imshow(img)
        plt.show()
    except:
        import traceback
        traceback.print_exc()


from frozendict import frozendict
hashed = ['{}.npy'.format(str(hash(frozendict(v1_params)))) for v1_params in param_grid]


images = dict([(k, np.load('act_npy/'+k)) for k in os.listdir('act_npy')])
def load_img(fname):
    plt.imshow(images[fname], cmap='gray', vmin=0, vmax=1)
    plt.show()

for f in os.listdir():
    if f in hashed:
        print(f)


# for v1_params in tqdm.tqdm_notebook(ParameterGrid(param_grid)[:10]):
#     print(v1_params)
%matplotlib inline
from ipywidgets import interact, SelectionSlider, Layout
import os
wid = SelectionSlider(
    options=list(images.keys())[:100],
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)
wid.layout.width = '1000px'

plt.figure(figsize=(3, 3))
plt.imshow(generate_horizontal_bar(96), cmap='gray', vmin=0, vmax=1)
interact(load_img, fname=wid)


